In [1]:
import torch
import transformers
from catalyst import dl
from src.runners import DistilMLMRunner
# from src.models import DistilpegasusStudentModel, PegasusForMLM #BertForMLM  DistilbertStudentModel
from catalyst.core import MetricAggregationCallback
from torch.utils.data import DataLoader
from src.callbacks import (
    CosineLossCallback,
    KLDivLossCallback,
    MaskedLanguageModelCallback,
    MSELossCallback,
    PerplexityMetricCallbackDistillation,
)
import pandas as pd

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:572: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (np.object, string),
/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:573: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (np.bool, bool),
/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/t

In [14]:
from datasets import load_dataset
PATH_TO_YOUR_DATASET = "./data"
dataset_1 = load_dataset("cnn_dailymail", '3.0.0')
# dataset = load_dataset('csv', data_files = {'train':["./modified_train.csv"], 'valid':["./modified_valid.csv"]})
# dataset = load_dataset('csv', data_files = {'train':[f"{PATH_TO_YOUR_DATASET}/train.csv"], 'valid':[f"{PATH_TO_YOUR_DATASET}/valid.csv"]})
# dataset_valid = load_dataset('csv', data_files = f"{PATH_TO_YOUR_DATASET}/valid.csv")
# dataset = load_dataset("cnn_dailymail")

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

Downloading: 159MB [00:34, 4.63MB/s]
Downloading: 376MB [01:21, 4.60MB/s]
Downloading: 2.11MB [00:02, 1.04MB/s]
Downloading: 46.4MB [01:21, 572kB/s]
Downloading: 2.43MB [00:02, 1.03MB/s]
Dataset cnn_dailymail downloaded and prepared to /home/ibrahim/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0a01b1abede4f646130574f203de57a293ded8a7a11e3406a539453afdfeb2c0. Subsequent calls will reuse this data.


In [34]:
from transformers.tokenization_utils.PreTrainedTokenizer import prepare_for_tokenization
prepare_for_tokenization(dataset_1['train'])

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



ModuleNotFoundError: No module named 'transformers.tokenization_utils.PreTrainedTokenizer'; 'transformers.tokenization_utils' is not a package

In [48]:
dataset_1['train']['article']

emmed in by the state legislature and bureaucracy. Palin is impulsive. Her charisma is such that she does not need to hold an office to command attention or wield influence. She resigned from the Alaska Oil and Gas Conservation Commission all of a sudden in 2004, plunged into a Republican gubernatorial primary in October 2005 and joined McCain\'s campaign without hesitation. Two of these three dizzying moves ended up in victory, and one did not. Two out of three isn\'t bad. Why shouldn\'t Palin think another gamble might pay off? Palin herself may not know her next move. Speculation about her presidential ambitions is premature, though it will be much easier for her to build a national organization now that she has no professional ties to Alaska. Whatever she does will be noticed, that\'s for sure. Because the attention lavished on Palin\'s decision is further evidence of her unwitting ability to bring out deep-seated feelings of admiration -- and loathing -- in people. We will be hear

In [2]:
import numpy as np
import datasets
# dataset_1.save_to_disk('cnn_dataset')
data = datasets.load_from_disk('cnn_dataset')
# np.save("cnn_dataset.npy", dataset_1)
# np.save("valid_dataset.npy", valid_dataset)
# train_dataset = np.load('cnn_dataset.npy', allow_pickle=True)
# valid_dataset = np.load('valid_dataset.npy', allow_pickle=True)

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/datasets/dataset_dict.py:675: ResourceWarning:

unclosed file <_io.TextIOWrapper name='/home/ibrahim/projects_dsp/konafa/cnn_dataset/dataset_dict.json' encoding='utf-8'>



In [5]:
tokenizer = AutoTokenizer.from_pretrained("tuner007/pegasus_paraphrase")

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:308: ResourceWarning:

unclosed file <_io.BufferedReader name='/home/ibrahim/.cache/huggingface/transformers/b3949a7257f9b4eaaf4f7785be079d89e4fec7d1c3763a58e6a2743635877d1f.1acf68c74589da6c7fa3548093824dfc450a54637f4356929bbfea7e294a68f8'>



In [7]:
tokenizer

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



PreTrainedTokenizerFast(name_or_path='tuner007/pegasus_paraphrase', vocab_size=96103, model_max_len=60, is_fast=True, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask_2>', 'additional_special_tokens': ['<mask_1>', '<unk_2>', '<unk_3>', '<unk_4>', '<unk_5>', '<unk_6>', '<unk_7>', '<unk_8>', '<unk_9>', '<unk_10>', '<unk_11>', '<unk_12>', '<unk_13>', '<unk_14>', '<unk_15>', '<unk_16>', '<unk_17>', '<unk_18>', '<unk_19>', '<unk_20>', '<unk_21>', '<unk_22>', '<unk_23>', '<unk_24>', '<unk_25>', '<unk_26>', '<unk_27>', '<unk_28>', '<unk_29>', '<unk_30>', '<unk_31>', '<unk_32>', '<unk_33>', '<unk_34>', '<unk_35>', '<unk_36>', '<unk_37>', '<unk_38>', '<unk_39>', '<unk_40>', '<unk_41>', '<unk_42>', '<unk_43>', '<unk_44>', '<unk_45>', '<unk_46>', '<unk_47>', '<unk_48>', '<unk_49>', '<unk_50>', '<unk_51>', '<unk_52>', '<unk_53>', '<unk_54>', '<unk_55>', '<unk_56>', '<unk_57>', '<unk_58>', '<unk_59>', '<unk_60>', '<unk_61>',

In [6]:
PATH_TO_YOUR_DATASET = "./data"
train_df = pd.read_csv(f"{PATH_TO_YOUR_DATASET}/train.csv")
valid_df = pd.read_csv(f"{PATH_TO_YOUR_DATASET}/valid.csv")


In [7]:

# valid_df.shape
valid_df = valid_df.drop('Unnamed: 0', axis=1)
train_df = train_df.drop('Unnamed: 0', axis=1)
train_df = train_df.dropna()
valid_df = valid_df.dropna()
valid_df.head(5)

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,text,headline
0,Receiving surgery to increase the volume of y...,\nIdentify the benefits and drawbacks of hair ...
1,"If you are building a new shed, it is importa...","\nBuild the shed up off of the ground.,\nPaint..."
2,Begin by rubbing the scratched wooden item wi...,"\nRub the walnut in.,\nRub the walnut across t..."
3,Fraud is defined as an intentional misreprese...,"\nIdentify fraud.,\nWrite down relevant inform..."
4,Gmail supports instant messaging over AIM or ...,\nUse Google Talk if you have a Google email a...


In [11]:
train_df.to_csv('modified_train.csv', index=False)
valid_df.to_csv('modified_valid.csv', index=False)

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [4]:
from typing import Iterable, Union

# import torch
from torch.utils.data import Dataset
from tqdm.auto import tqdm
# import transformers
from transformers import AutoTokenizer


class LanguageModelingDataset(Dataset):
    """
    Dataset for (masked) language model task.
    Can sort sequnces for efficient padding.
    """

    def __init__(
        self,
        texts: Iterable[str],
        tokenizer: Union[
            str, transformers.models.pegasus.tokenization_pegasus_fast.PegasusTokenizerFast
        ],
        max_seq_length: int = None,
        sort: bool = True,
        lazy: bool = False,
    ):
        """
        Args:
            texts (Iterable): Iterable object with text
            tokenizer (str or tokenizer): pre trained
                huggingface tokenizer or model name
            max_seq_length (int): max sequence length to tokenize
            sort (bool): If True then sort all sequences by length
                for efficient padding
            lazy (bool): If True then tokenize and encode sequence
                in __getitem__ method
                else will tokenize in __init__ also
                if set to true sorting is unavialible
        """
        if sort and lazy:
            raise Exception(
                "lazy is set to True so we can't sort"
                " sequences by length.\n"
                "You should set sort=False and lazy=True"
                " if you want to encode text in __get_item__ function"
            )
        if isinstance(tokenizer, str):
            self.tokenizer = tokenizer
        elif isinstance(
            tokenizer, transformers.models.pegasus.tokenization_pegasus_fast.PegasusTokenizerFast
            # transformers.tokenization_utils.PreTrainedTokenizer
        ):
            self.tokenizer = tokenizer
        else:
            raise TypeError(
                "tokenizer argument should be a model name"
                + " or huggingface PreTrainedTokenizer"
            )
            # self.tokenizer = tokenizer

        self.max_seq_length = max_seq_length

        self.lazy = lazy

        if lazy:
            self.texts = texts

        if not lazy:
            pbar = tqdm(texts, desc="tokenizing texts")
            self.encoded = [
                self.tokenizer.encode(text, max_length=max_seq_length)
                for text in pbar
            ]
            if sort:
                self.encoded.sort(key=len)

        self.length = len(texts)

        self._getitem_fn = (
            self._getitem_lazy if lazy else self._getitem_encoded
        )

    def __len__(self):
        """Return length of dataloader"""
        return self.length

    def _getitem_encoded(self, idx) -> torch.Tensor:
        return torch.tensor(self.encoded[idx])

    def _getitem_lazy(self, idx) -> torch.Tensor:
        encoded = self.tokenizer.encode(
            self.texts[idx], max_length=self.max_seq_length
        )

        return torch.tensor(encoded)

    def __getitem__(self, idx):
        """Return tokenized and encoded sequence"""
        return self._getitem_fn(idx)

In [4]:
# from src.data import LanguageModelingDataset
from transformers import AutoTokenizer
from transformers.data.data_collator import DataCollatorForLanguageModeling

model_name = "tuner007/pegasus_paraphrase"

tokenizer = AutoTokenizer.from_pretrained("tuner007/pegasus_paraphrase")
print('this is type of tokinzer ======== ',type(tokenizer))

train_dataset = LanguageModelingDataset(train_df["text"].tolist(), tokenizer)
valid_dataset = LanguageModelingDataset(valid_df["text"].tolist(), tokenizer)


/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:308: ResourceWarning:

unclosed file <_io.BufferedReader name='/home/ibrahim/.cache/huggingface/transformers/b3949a7257f9b4eaaf4f7785be079d89e4fec7d1c3763a58e6a2743635877d1f.1acf68c74589da6c7fa3548093824dfc450a54637f4356929bbfea7e294a68f8'>

tokenizing texts: 100%|██████████| 4/4 [00:00<00:00, 5491.72it/s]this is type of tokinzer ========  <class 'transformers.models.pegasus.tokenization_pegasus_fast.PegasusTokenizerFast'>



In [3]:
import numpy as np
# np.save("train_dataset.npy", train_dataset)
# np.save("valid_dataset.npy", valid_dataset)
train_dataset = np.load('train_dataset.npy', allow_pickle=True)
valid_dataset = np.load('valid_dataset.npy', allow_pickle=True)

In [4]:
from  transformers.data.data_collator import default_data_collator
collate_fn = default_data_collator
train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn, batch_size=2
)
valid_dataloader = DataLoader(
    valid_dataset, collate_fn=collate_fn, batch_size=2
)
loaders = {"train": train_dataloader, "valid": valid_dataloader}

In [10]:
print(train_dataloader.batch_size)
for i, batch in enumerate(train_dataloader):
    b = next(iter(batch.values()))

2
/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



StopIteration: 

In [2]:
from torch import nn
import transformers


class PegasusForMLM(nn.Module):
    """
    BertForMLM

    Simplified huggingface model
    """

    def __init__(
        self,
        model_name: str = "tuner007/pegasus_paraphrase",
        output_logits: bool = True,
        output_hidden_states: bool = True,
    ):
        """
        Args:
            model_name: huggingface model name
            output_logits: same as in huggingface
            output_hidden_states: same as in huggingface
        """
        super().__init__()
        self.config = transformers.AutoConfig.from_pretrained(
            model_name,
            output_hidden_states=output_hidden_states,
            output_logits=output_logits
        )
        self.bert = transformers.PegasusForCausalLM.from_pretrained(
            model_name, config=self.config
        )

    def forward(self, *model_args, **model_kwargs):
        """Forward method"""
        return self.bert(*model_args, **model_kwargs)


/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [3]:
from typing import Dict, List

class DistilpegasusStudentModel(nn.Module):
    """
    DistilbertStudentModel

    Distil model class based on huggingface class but with
    initialization in it. Model will take vocabulary
    layers from specified teacher model
    """

    def __init__(
        self,
        teacher_model_name: str = "tuner007/pegasus_paraphrase",
        layers: List[int] = None,
        extract: bool = True,
    ):
        """
        Args:
            teacher_model_name: name of the model to distil
            layers: layers indexes to initialize
            extract: bool flag, if you want to initialize your model with
                layers of the teacher model then set this to true
        """
        super().__init__()
        if layers is None:
            layers = [0, 2, 4, 7, 9, 11, 15]
        teacher_config = transformers.AutoConfig.from_pretrained(
            teacher_model_name, output_hidden_states=True, output_logits=True
        )
        teacher = transformers.PegasusForCausalLM.from_pretrained(
            teacher_model_name, config=teacher_config
        )
        distil_sd = None
        if extract:
            distil_sd = self._extract(teacher, layers)
        if teacher_model_name == "tuner007/pegasus_paraphrase":
            f = open('a7ooo.txt', 'a')
            f.write(','.join(map(str, distil_sd)))
            f.close()
            student_config = transformers.AutoConfig.from_pretrained(
                "tuner007/pegasus_paraphrase",
                output_hidden_states=True,
                output_logits=True,
            )
            self.student = transformers.PegasusForCausalLM.from_pretrained(
                "tuner007/pegasus_paraphrase",
                config=student_config,
                state_dict=distil_sd,
            )
        elif teacher_model_name == "bert-base-cased":
            student_config = transformers.AutoConfig.from_pretrained(
                "distilbert-base-cased",
                output_hidden_states=True,
                output_logits=True,
            )
            self.student = transformers.DistilBertForMaskedLM.from_pretrained(
                "distilbert-base-cased",
                config=student_config,
                state_dict=distil_sd,
            )
        else:
            student_config = transformers.AutoConfig.from_pretrained(
                "distilbert-base-multilingual-cased",
                output_hidden_states=True,
                output_logits=True,
            )
            self.student = transformers.DistilBertForMaskedLM.from_pretrained(
                "distilbert-base-multilingual-cased",
                config=student_config,
                state_dict=distil_sd,
            )

    def forward(self, *model_args, **model_kwargs):
        """Forward nethod"""
        return self.student(*model_args, **model_kwargs)

    @staticmethod
    def _extract(
        teacher_model,
        layers: List[int],
        prefix_teacher: str = "model",
        prefix_student: str = "pegasus_student",
    ) -> Dict[str, torch.Tensor]:
        """
        Extracts state dict from teacher model

        Args:
            teacher_model: model to extract
            layers: layers indexes to initialize
            prefix_teacher: name of the teacher model
            prefix_student: name of the student model
        """
        state_dict = teacher_model.state_dict()
        # f = open('state_dict.txt', 'a')
        # f.write(','.join(map(str, teacher_model.state_dict())))
        # f.close()
        compressed_sd = {}

        # extract embeddings
        for w in ["embed_tokens", "embed_positions"]:
            compressed_sd[
                f"{prefix_student}.decoder.{w}.weight"
            ] = state_dict[f"{prefix_teacher}.decoder.{w}.weight"]
        print('finished_11111111')
        for w in ["weight", "bias"]:
            compressed_sd[
                f"{prefix_student}.decoder.layers.0.self_attn_layer_norm.{w}"
            ] = state_dict[f"{prefix_teacher}.decoder.layers.0.self_attn_layer_norm.{w}"]
        print('finished_222222')
        # extract encoder
        std_idx = 0
        for teacher_idx in layers:
            for w in ["weight", "bias"]:
                compressed_sd[
                    f"{prefix_student}.transformer.layer.{std_idx}.attention.q_lin.{w}"  # noqa: E501
                ] = state_dict[
                    f"{prefix_teacher}.decoder.layers.{teacher_idx}.encoder_attn.q_proj.{w}"  # noqa: E501
                ]
                compressed_sd[
                    f"{prefix_student}.transformer.layer.{std_idx}.attention.k_lin.{w}"  # noqa: E501
                ] = state_dict[
                    f"{prefix_teacher}.decoder.layers.{teacher_idx}.encoder_attn.k_proj.{w}"  # noqa: E501
                ]
                compressed_sd[
                    f"{prefix_student}.transformer.layer.{std_idx}.attention.v_lin.{w}"  # noqa: E501
                ] = state_dict[
                    f"{prefix_teacher}.decoder.layers.{teacher_idx}.encoder_attn.v_proj.{w}"  # noqa: E501
                ]

                compressed_sd[
                    f"{prefix_student}.transformer.layer.{std_idx}.attention.out_lin.{w}"  # noqa: E501
                ] = state_dict[
                    f"{prefix_teacher}.decoder.layers.{teacher_idx}.self_attn.out_proj.{w}"  # noqa: E501
                ]
                compressed_sd[
                    f"{prefix_student}.transformer.layer.{std_idx}.sa_layer_norm.{w}"  # noqa: E501
                ] = state_dict[
                    f"{prefix_teacher}.decoder.layers.{teacher_idx}.self_attn_layer_norm.{w}"  # noqa: E501
                ]

                compressed_sd[
                    f"{prefix_student}.transformer.layer.{std_idx}.ffn.lin1.{w}"  # noqa: E501
                ] = state_dict[
                    f"{prefix_teacher}.decoder.layers.{teacher_idx}.fc1.{w}"  # noqa: E501
                ]
                compressed_sd[
                    f"{prefix_student}.transformer.layer.{std_idx}.ffn.lin2.{w}"  # noqa: E501
                ] = state_dict[
                    f"{prefix_teacher}.decoder.layers.{teacher_idx}.fc2.{w}"  # noqa: E501
                ]
                compressed_sd[
                    f"{prefix_student}.transformer.layer.{std_idx}.output_layer_norm.{w}"  # noqa: E501
                ] = state_dict[
                    f"{prefix_teacher}.decoder.layers.{teacher_idx}.final_layer_norm.{w}"  # noqa: E501
                ]

            std_idx += 1
        print('finished_3333333333')
        # extract vocab
        compressed_sd[f"lm_head.weight"] = state_dict[
            f"lm_head.weight"
        ]
        # compressed_sd[f"vocab_projector.bias"] = state_dict[
        #     f"cls.predictions.bias"
        # ]

        # for w in ["weight", "bias"]:
        #     compressed_sd[f"vocab_transform.{w}"] = state_dict[
        #         f"cls.predictions.transform.dense.{w}"
        #     ]
        #     compressed_sd[f"vocab_layer_norm.{w}"] = state_dict[
        #         f"cls.predictions.transform.LayerNorm.{w}"
        #     ]

        return compressed_sd

In [9]:
# model_name = "tuner007/pegasus_paraphrase"
# teacher = PegasusForMLM(model_name)
# student = DistilpegasusStudentModel(model_name)

teacher = torch.load('pg.pt')
student = torch.load('st_3dec_3enc.pt')

model = torch.nn.ModuleDict({"teacher": teacher, "student": student})

callbacks = {
    "masked_lm_loss": MaskedLanguageModelCallback(),
    "mse_loss": MSELossCallback(),
    "cosine_loss": CosineLossCallback(),
    "kl_div_loss": KLDivLossCallback(),
    "loss": MetricAggregationCallback(
        prefix="loss",
        mode="weighted_sum",
        metrics={
            "cosine_loss": 1.0,
            "masked_lm_loss": 1.0,
            "kl_div_loss": 1.0,
            "mse_loss": 1.0
        }
    ),
    "optimizer": dl.OptimizerCallback(),
    "perplexity": PerplexityMetricCallbackDistillation()
}

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [6]:
model_name = "tuner007/pegasus_paraphrase"
teacher = PegasusForMLM(model_name)
student = DistilpegasusStudentModel(model_name)

ormer.layer.5.ffn.lin2.bias', 'pegasus_student.transformer.layer.5.output_layer_norm.bias', 'pegasus_student.transformer.layer.6.attention.q_lin.weight', 'pegasus_student.transformer.layer.6.attention.k_lin.weight', 'pegasus_student.transformer.layer.6.attention.v_lin.weight', 'pegasus_student.transformer.layer.6.attention.out_lin.weight', 'pegasus_student.transformer.layer.6.sa_layer_norm.weight', 'pegasus_student.transformer.layer.6.ffn.lin1.weight', 'pegasus_student.transformer.layer.6.ffn.lin2.weight', 'pegasus_student.transformer.layer.6.output_layer_norm.weight', 'pegasus_student.transformer.layer.6.attention.q_lin.bias', 'pegasus_student.transformer.layer.6.attention.k_lin.bias', 'pegasus_student.transformer.layer.6.attention.v_lin.bias', 'pegasus_student.transformer.layer.6.attention.out_lin.bias', 'pegasus_student.transformer.layer.6.sa_layer_norm.bias', 'pegasus_student.transformer.layer.6.ffn.lin1.bias', 'pegasus_student.transformer.layer.6.ffn.lin2.bias', 'pegasus_student.t

In [3]:
# torch.save(teacher, 'pg.pt')
teacher = torch.load(teacher, 'pg.pt')
# torch.save(student, 'pg_student.pt')

NameError: name 'teacher' is not defined

In [4]:
teacher = torch.load('pg.pt')
student = torch.load('st_3dec_3enc.pt')

In [5]:
train_dataset = torch.load("train_dataset1.pt")

In [13]:
batch = train_dataset['input_ids'][0]
attenion_mask = train_dataset[0].attention_mask

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [31]:
teacher.bert.model(torch.reshape(torch.tensor(batch), (1,130)), torch.tensor(attenion_mask))

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



ValueError: not enough values to unpack (expected 2, got 1)

In [32]:
teacher.bert.model

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



PegasusDecoderWrapper(
  (decoder): PegasusDecoder(
    (embed_tokens): Embedding(96103, 1024, padding_idx=0)
    (embed_positions): PegasusSinusoidalPositionalEmbedding(60, 1024)
    (layers): ModuleList(
      (0): PegasusDecoderLayer(
        (self_attn): PegasusAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (encoder_attn): PegasusAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_featur

In [25]:
torch.tensor(batch)

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



torch.Size([130])

In [26]:
torch.reshape(torch.tensor(batch), (1,130))

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



tensor([[14937,  1034,   116,   582,   728, 10493, 47026,   141,   114,  1093,
          1768,   107, 73032,  1722,  2729,   233,  1191, 23887,   121,  1768,
           107, 73032,  1722,   214,   233,   551, 23887,   121,   115,   109,
         61118,  2349,   640,   112, 19350, 12906,  1068,   110,   108,  3432,
          1291, 42993,  2375,   110,   107,     1,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [6]:
# print(torch.cuda.current_device())
# print(torch.cuda.device(0))
# print()
state_dict = student.state_dict()
f = open('state_dict_student.txt', 'a')
f.write(','.join(map(str, student.state_dict())))
f.close()

In [11]:
runner = DistilMLMRunner(device=torch.device("cuda"))
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
runner.train(
    model=model,
    optimizer=optimizer,
    loaders=loaders,
    verbose=True,
    check=True,
    callbacks=callbacks,
)


1/1 * Epoch (train):   0% 0/85711 [00:00<?, ?it/s]

StopIteration: 

In [61]:
# torch.cuda.empty_cache() 
torch.cuda.empty()

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



AttributeError: module 'torch.cuda' has no attribute 'empty'

In [3]:
dataset = datasets.load_from_disk('cnn_dataset')


tokenizer = AutoTokenizer.from_pretrained("tuner007/pegasus_paraphrase")
train_dataset = tokenizer(dataset['train']['article'],padding='longest')
valid_dataset = tokenizer(dataset['validation']['article'],padding='longest')
train_label = tokenizer(dataset['train']['highlights'],padding='longest')
valid_label = tokenizer(dataset['validation']['highlights'],padding='longest')
torch.save(train_dataset,'train_dataset1.pt')
torch.save(valid_dataset,'valid_dataset1.pt')
torch.save(train_label,'train_label1.pt')
torch.save(valid_label,'valid_label1.pt')
# train_dataset = LanguageModelingDataset(dataset['train']['article'][0:10000], tokenizer)
# valid_dataset = LanguageModelingDataset(dataset['validation']['article'][0:10000], tokenizer)
# train_label = LanguageModelingDataset(dataset['train']['highlights'][0:10000], tokenizer)
# valid_label = LanguageModelingDataset(dataset['validation']['highlights'][0:10], tokenizer)


NameError: name 'AutoTokenizer' is not defined

In [9]:
np.save('train_dataset1.npy',train_dataset)
np.save('valid_dataset1.npy',valid_dataset)
np.save('train_label1.npy',train_label)
np.save('valid_label1.npy',valid_dataset)

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/numpy/core/_asarray.py:171: FutureWarning:

The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.

/home/ibrahim/anaconda3/envs/hug/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



In [24]:
train_dataset[0]

tensor([  143, 40155,   158,  1315, 12746,  1017,   112,   736,  2899,  1069,
          694,   121, 18707,   428,   121,  6011, 36496,   112,  1348,  3637,
          134,   109, 34372, 14367,   894,  3164,   107,  1969,   224,   109,
         2808,   607,   112,   236,   199,   109, 36496,   138,   286,   107,
            1])

In [19]:
len(dataset['train']['article'][0])

9396

In [1]:
from datasets import load_dataset
dataset = load_dataset("gigaword")

Downloading: 4.46kB [00:00, 3.86MB/s]                   
Downloading: 2.16kB [00:00, 1.67MB/s]                   
Using custom data configuration default
Downloading: 578MB [02:51, 3.38MB/s]
Dataset gigaword downloaded and prepared to /home/ibrahim/.cache/huggingface/datasets/gigaword/default/1.2.0/c518c578e42a6afe842b09e979ee2907ea42a12b57ba992fae9e9d7347825245. Subsequent calls will reuse this data.


In [2]:
dataset.save_to_disk('gigaword')